# -*- coding: utf-8 -*-
#  Copyright 2025 -  United Kingdom Research and Innovation
#
#  Licensed under the Apache License, Version 2.0 (the "License");
#  you may not use this file except in compliance with the License.
#  You may obtain a copy of the License at
#
#      http://www.apache.org/licenses/LICENSE-2.0
#
#  Unless required by applicable law or agreed to in writing, software
#  distributed under the License is distributed on an "AS IS" BASIS,
#  WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
#  See the License for the specific language governing permissions and
#  limitations under the License.
#
#  Authored by:    Laura Murgatroyd (STFC-UKRI)
#                  Franck Vidal (STFC-UKRI)
#                  Gemma Fardell (STFC-UKRI)

# Flexible Geometry

This notebook introduces the `Cone3D_Flex` `AcquisitionGeometry` which allows setting a different source and detector position for each acquired radiograph.

Learning objectives:
- Create a `Cone3D_Flex` `AcquisitionGeometry`
- Reconstruct using FDK from ASTRA
- Compare the forward projections to the radiographs
- Reconstruct using SIRT

In [ ]:
from cil.io import TIFFStackReader
from cil.utilities.display import show2D, show_SOUV_geometry_vectors, show_geometry
from cil.framework import ImageGeometry, AcquisitionGeometry, AcquisitionData
import os
from cil.utilities.jupyter import islicer
import matplotlib
import numpy as np
from cil.plugins.astra import FBP
from cil.processors import TransmissionAbsorptionConverter

## Load the Radiographs

The data consists of TIFF files - the projections, and a CSV file which contains the source and detector positions for each angle

In [ ]:
filepath=r"C:\Users\lhe97136\Work\GitHub\non-standard-CT\output\non-standard-trajectories\projections"


First let's load and view the projections

In [ ]:
projection_array = TIFFStackReader(filepath).read()

In [ ]:
show2D([projection_array]*2, slice_list=[0,100])

In [ ]:
print(projection_array.shape)

### Display the Sinogram

In [ ]:
show2D([projection_array], slice_list=(1,80), title='Projection Array')

Let's examine the projections using islicer - do you see anything unusual?

In [ ]:
islicer(projection_array, title='Projection Array')

It looks like the dragon is bouncing up and down! This is because some or all of the following are varying between each radiograph:
- detector position
- detector angle
- source position

The data came with a CSV file which describes this geometry for each radiograph.

# Read the Geometry Information

Now we'll read the information from the csv file:

In [ ]:
csv_filepath = os.path.join(filepath, 'geom.csv')

column_names = ["fname",
    "source position (x)", "source position (y)", "source position (z)",
    "imager centre (x)", "imager centre (y)", "imager centre (z)",
    "imager u vector (x)", "imager u vector (y)", "imager u vector (z)",
    "imager v vector (x)", "imager v vector (y)", "imager v vector (z)",
    "angle",
];

# fname,source position (x),source position (y),source position (z),imager centre (x),imager centre (y),imager centre (z),imager u vector (x),imager u vector (y),imager u vector (z),imager v vector (x),imager v vector (y),imager v vector (z),angle

# read the csv file:
import pandas as pd
df = pd.read_csv(csv_filepath)

source_position_set = df[['source position (x)', 'source position (y)', 'source position (z)']].values
detector_position_set = df[['imager centre (x)', 'imager centre (y)', 'imager centre (z)']].values
detector_direction_x_set = df[['imager u vector (x)', 'imager u vector (y)', 'imager u vector (z)']].values
detector_direction_y_set = df[['imager v vector (x)', 'imager v vector (y)', 'imager v vector (z)']].values
angles = df['angle'].values


# Create a CIL Acquisition Geometry

To define the acquisition geometry in CIL, we create a Cone3D_Flex Acquisition Geometry.

This requires us to set:
- `source_position_set` - This is a list of 3D vectors describing the position of the source for each radiograph acquired.
- `detector_position_set` - This is a list of 3D vectors describing the position of the detector for each radiograph acquired.
- `detector_direction_x_set` - This is a list of 3D vectors describing the direction of the detector_x
- `detector_direction_y_set` - This is a list of 3D vectors describing the direction of the detector_y

We have read all of these from the csv in the cell above!

In [ ]:
acq_geometry = AcquisitionGeometry.create_Cone3D_SOUV(source_position_set, detector_position_set,
                                                detector_direction_x_set, detector_direction_y_set)


Note: we could also have set the `volume_centre_position`. This is a 3D vector describing the position of the centre of the reconstructed volume (x,y,z). We have not set this, which means it will be set to the default of [0,0,0]: the origin.

As with other geometry types in CIL, we also need to set the panel size, pixel size, and data labels. Printing the data shape and examining the radiographs we showed above helps us with this:

In [ ]:
print(projection_array.shape)

We can see that we have 500 angles and our panel is 160x160

TODO: how do we explain where we get the pixel size from? We got it from gvxr originally
pixel_size = (0.25, 0.25)

In [ ]:
number_of_pixels = projection_array.shape[1:3]
pixel_size = (0.25, 0.25)

In [ ]:
acq_geometry.set_panel(number_of_pixels, pixel_size)
acq_geometry.set_labels(['angle','vertical','horizontal'])

The standard CIL definitions of axes are shown in this image:

![title](images/07_cone_geometry_example.png)

This display tool allows us to visualise the motion of the source and detector throughout the scan:

In [ ]:
show_SOUV_geometry_vectors(acq_geometry);

If we print the acquisition geometry, we can also see the first few source and detector positions, and detector vectors:

In [ ]:
print(acq_geometry)

TODO: add comment about show_geometry not used for this geom type

# Reconstruct with ASTRA

Now we can create our CIL AcquisitionData as normal. We'll be reconstructing with ASTRA so we also reorder the data to enable this:

In [ ]:
acq_data = AcquisitionData(projection_array, geometry=acq_geometry)
acq_data.reorder(order='astra')

We need to convert to absoprtion data first:

In [ ]:
absorp_data = TransmissionAbsorptionConverter()(acq_data)

In [ ]:
show2D(absorp_data.array[:,0])

Reconstructing requires setting an image geometry. This is the description of the reconstruction volume. We can get a default image geometry from our AcquisitionData:

In [ ]:
image_geometry = absorp_data.geometry.get_ImageGeometry()
print(image_geometry)

The default ImageGeometry is constructed using an average of the magnification values across all of the projectons.

In [ ]:
fbp = FBP(absorp_data.geometry.get_ImageGeometry(), absorp_data.geometry) 
fbp.set_input(absorp_data)

In [ ]:
recon = fbp.get_output()

show2D(recon, title='Reconstruction')

In [ ]:

from cil.plugins.astra import ProjectionOperator


PO = ProjectionOperator( recon.geometry, absorp_data.geometry)

forward_projection = PO.direct(recon)

In [ ]:
show2D([forward_projection.array[:,100,:], absorp_data.array[:,100,:]], title=["Forward Proj 10", "Proj 10"])

# SIRT

In [ ]:
from cil.optimisation.algorithms import CGLS, SIRT

x0 = recon.geometry.copy().allocate(0)

sirt = SIRT(initial=x0, operator=PO, data=absorp_data)
sirt.update_objective_interval = 10
sirt.run(100)

recon_sirt = sirt.solution

In [ ]:
show2D(recon_sirt, title='SIRT Reconstruction')

In [ ]:
show2D([recon, recon_sirt], title=['FBP Reconstruction', 'SIRT Reconstruction'])